In [1]:
import polars as pl
import pandas as pd

import sys
sys.path.insert(0, 'C:\\Users\\rockh\\Repositories\\Tsuro\\')

from tsuro.datasets import load_cba_trades


In [4]:
trades = load_cba_trades(lazy=False)

In [9]:
trades.select("Time", "")

Time
str
"""6/10/2021 17:00"""
"""6/10/2021 16:54"""
"""6/10/2021 16:47"""
"""6/10/2021 16:46"""
"""6/10/2021 16:45"""
…
"""6/10/2021 7:05"""
"""6/10/2021 7:05"""
"""6/10/2021 7:05"""
